#  Многомерная интерполяция на регулярной сетке

# Задание
Задана таблица значений функции вида x, y, z(x, y). С помощью интерполяции, используя
метод полинома Ньютона, найти приближённое значение функции от введённых 𝑋  Y.

Ввод: значение 𝑋, Y и степени полиномов n_x, n_y

Вывод: значение функции от 𝑋 Y
    

# Описание алгоритма построения интерполяционного полинома Ньютона
Для двух точек: $$y(x_i, x_j) ={ y_i - y_j \over x_i - x_j}$$

Для трёх точек: $$y(x_i, x_j, x_k) ={ y(x_i, x_j) - y(x_j, x_k) \over x_i - x_k}$$

Для n точек: $$y(x_i, x_j, ..., x_n) ={ y(x_i, x_j, ..., x_{n - 1}) - y(x_j, ..., x_n) \over x_i - x_n}$$

Отсюда искомый полином будет равен:
$$P_n(x) = y_0 + (x-x_0)y(x_0, x_1) + (x - x_0)(x - x_1)y(x_0, x_1, x_2) + ... \\ + (x - x_0)(x - x_1)...(x - x_{n - 1})y(x_0, x_1, ..., x_n)$$
$$P_n(x) = y_0 + \sum\limits_{k = 0}^n (x - x_0) ...(x - x_{n - 1})y(x_0,x_1,...,x_k)$$

# Описание алгоритма  интерполяции для двух аргументов
Найти z(x, y)

1) находим z(x, $y_k$) для всех k, где 0 $\leq$ k  $\leq$ n_y. Результат каждого вычисления поочередно сохраняем как $t_k$

2) Находим z(y, t). Это и есть искомый результат.

При таком алгоритме требуется запустить алгоритм интерполяционного алгоритма Ньютона n_y + 1 раз.

In [8]:
import numpy as np
from heapq import nsmallest

In [9]:
LEN_ERROR = 2

In [10]:
#функция для отладки
def func(x, y):
    return x**2 + y**2

In [11]:
# функция для отладки
def make_file(start_x, stop_x, step_x, start_y, stop_y, step_y):
    f = open('lab2.txt', 'w')

    num_x = abs((stop_x - start_x) / step_x)
    num_y = abs((stop_y - start_y) / step_y)

    matrix = np.zeros((int(num_x + 2), int(num_y + 2)))
    x = start_x
    y = start_y
    for i in range(int(num_x) + 1):
        matrix[i + 1][0] = x
        x += step_x

    for i in range(int(num_y) + 1):
        matrix[0][i + 1] = y
        y += step_y

    for i in range(1, len(matrix)):
        for j in range(1, len(matrix[0])):
            matrix[i][j] = func(matrix[i][0], matrix[0][j])
    for i in range(len(matrix)):
        s = " ".join(str(x) for x in matrix[i])
        f.write(s + "\n")

    f.close()

In [12]:
make_file(0, 10, 0.5, 0, 10, 0.5)

In [13]:
def make_coordinates_table(filename):
    with open(filename) as f:
        coordinates = [list(map(float, row.split())) for row in f.readlines()]
    return coordinates

In [14]:
def make_table(coordinates, n_x, n_y, x, y):
    if n_x >= len(coordinates) - 1 or n_y >= len(coordinates[0]) - 1:
        print("Can not do polynomial")
        return LEN_ERROR, LEN_ERROR
    x_list = list()
    for i in range(1, len(coordinates)):
        x_list.append([coordinates[i][0], int(i)])

    data_x = np.array(nsmallest(n_x + 1, x_list, key=lambda row: abs(row[0] - x)))
    data_x = data_x[data_x[:, 0].argsort()]

    y_list = list()
    for i in range(1, len(coordinates[0])):
        y_list.append([coordinates[0][i], int(i)])

    data_y = np.array(nsmallest(n_y + 1, y_list, key=lambda row: abs(row[0] - y)))
    data_y = data_y[data_y[:, 0].argsort()]

    list_y = list()
    matrix = np.zeros((n_x + 1, n_y + 2))

    start_x_ind = min(data_x[0][1], data_x[n_x][1])
    start_y_ind = min(data_y[0][1], data_y[n_y][1])
    stop_x_ind = max(data_x[0][1], data_x[n_x][1])
    stop_y_ind = max(data_y[0][1], data_y[n_y][1])

    for i in range(int(start_y_ind), int(stop_y_ind) + 1):
        list_y.append(coordinates[0][i])

    temp_x_ind = int(start_x_ind)
    for i_matr in range(len(matrix)):
        matrix[i_matr][0] = coordinates[temp_x_ind][0]
        j_matr = 1
        for j_coord in range(int(start_y_ind), int(stop_y_ind) + 1):
            matrix[i_matr][j_matr] = coordinates[temp_x_ind][j_coord]
            j_matr += 1
        temp_x_ind += 1
        if (temp_x_ind > stop_x_ind):
            break
    return matrix, list_y

In [15]:
def fill_table(table, n):
    step = 1
    temp_column = 2
    for count in range(n + 1):
        temp_row = 0
        while (temp_row + step) != len(table):
            table[temp_row][temp_column] = ((table[temp_row][temp_column - 1]
                                           - table[temp_row + 1][temp_column - 1]) * 1.0 /
                                            (table[temp_row][0] - table[temp_row + step][0]))
            temp_row += 1
        step += 1
        temp_column += 1
    return table

In [16]:
def get_polynomial(x, table, n):
    result = table[0][1]
    temp_column = 2
    for i in range (n):
        term = 1
        for j in range (temp_column - 1):
            term *= (x - table[j][0])
        term *= table[0][temp_column]
        result += term
        temp_column += 1
    return result

In [17]:
def newton(x, n, coordinates):
    table = np.zeros((n + 1, n))
    table = np.append(coordinates, table, axis=1)
    fill_table(table, n)
    result = get_polynomial(x, table, n)
    return result

In [18]:
def newton_for_2_args(data_matrix, list_y, x, y, n_x, n_y):
    list_result = np.zeros((n_y + 1, 2))
    temp_matr = np.zeros((len(data_matrix), 2))

    for i in range(len(list_result)):
        list_result[i][0] = list_y[i]

    for i in range(len(data_matrix)):
        temp_matr[i][0] = data_matrix[i][0]

    temp_y = 0
    while(temp_y < len(list_result)):
        for row in range(n_x + 1):
            temp_matr[row][1] = data_matrix[row][temp_y+ 1]
        list_result[temp_y][1] = newton(x, n_x, temp_matr)
        temp_y += 1
    result = newton(y, n_y, list_result)
    return result

In [19]:
def main(x, y, nx, ny):
    coordinates = make_coordinates_table("lab2.txt")
    data_matrix, list_y = make_table(coordinates, nx, ny, x, y)
    if type(data_matrix) == type(LEN_ERROR):
        return
    result = newton_for_2_args(data_matrix, list_y, x, y, nx, ny)
    print(result)

# Введите х, y и степени полиномов nx, ny

In [ ]:
coordinates = make_coordinates_table("lab2.txt")
for i in coordinates:
    print(i)

In [21]:
x = 3.7
y = 5.9
nx = 3
ny = 4
main(x, y, nx, ny)

48.5
